# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
df = pd.read_csv("../output_data/cities_550.csv")
del df["Unnamed: 0"]
df

,JSON,DATE,CITY,COUNTRY,LAT,LNG,MAX_TEMP,HUMIDITY,CLOUDINESS,WIND_SPEED
0,"{'coord': {'lon': -175.2, 'lat': -21.2}, 'weat...",2022-07-31 02:58:16,Vaini,TO,-21.2000,-175.2000,80.76,89.0,40.0,10.36
1,"{'coord': {'lon': -122.4286, 'lat': 37.4636}, ...",2022-07-31 02:58:18,Half Moon Bay,US,37.4636,-122.4286,62.33,87.0,100.0,16.11
2,"{'coord': {'lon': 168.3333, 'lat': -46.6}, 'we...",2022-07-31 02:58:19,Bluff,NZ,-46.6000,168.3333,47.80,77.0,49.0,16.26
3,"{'coord': {'lon': -79.2353, 'lat': 42.097}, 'w...",2022-07-31 02:58:21,Jamestown,US,42.0970,-79.2353,61.48,55.0,1.0,1.63
4,"{'coord': {'lon': -73.9662, 'lat': 42.6001}, '...",2022-07-31 02:57:37,Albany,US,42.6001,-73.9662,65.89,78.0,0.0,4.85
...,...,...,...,...,...,...,...,...,...,...
498,"{'coord': {'lon': -157.8378, 'lat': 21.4447}, ...",2022-07-31 03:15:04,‘Āhuimanu,US,21.4447,-157.8378,81.55,71.0,75.0,16.11
499,"{'coord': {'lon': 36.8595, 'lat': 49.4627}, 'w...",2022-07-31 03:15:06,Balakliya,UA,49.4627,36.8595,60.71,71.0,0.0,6.76
500,"{'coord': {'lon': -60.2177, 'lat': 46.2367}, '...",2022-07-31 03:15:09,Sydney Mines,CA,46.2367,-60.2177,64.26,94.0,75.0,10.36
501,"{'coord': {'lon': 44.8414, 'lat': 51.5506}, 'w...",2022-07-31 03:15:11,Lysyye Gory,RU,51.5506,44.8414,65.25,86.0,100.0,9.40


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key) 

In [7]:
locations = df[["LAT", "LNG"]]

weights = df["HUMIDITY"]

In [8]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [10]:
best_cities_df = df.loc[ (df["MAX_TEMP"].between(69, 84)) & (df["HUMIDITY"] < 80) & (df["CLOUDINESS"] < 50), :]
best_cities_df.dropna()
best_cities_df

,JSON,DATE,CITY,COUNTRY,LAT,LNG,MAX_TEMP,HUMIDITY,CLOUDINESS,WIND_SPEED
10,"{'coord': {'lon': -114.0611, 'lat': 27.9769}, ...",2022-07-31 02:58:37,Guerrero Negro,MX,27.9769,-114.0611,70.70,77.0,5.0,13.85
35,"{'coord': {'lon': -134.9692, 'lat': -23.1203},...",2022-07-31 02:59:30,Rikitea,PF,-23.1203,-134.9692,72.93,75.0,9.0,15.93
37,"{'coord': {'lon': -139.0333, 'lat': -9.8}, 'we...",2022-07-31 02:59:34,Atuona,PF,-9.8000,-139.0333,76.68,78.0,14.0,14.43
41,"{'coord': {'lon': -159.775, 'lat': -21.2078}, ...",2022-07-31 02:59:41,Avarua,CK,-21.2078,-159.7750,78.85,69.0,40.0,20.71
48,"{'coord': {'lon': -17.1, 'lat': 32.6667}, 'wea...",2022-07-31 02:59:55,Ponta do Sol,PT,32.6667,-17.1000,69.94,79.0,14.0,6.98
...,...,...,...,...,...,...,...,...,...,...
458,"{'coord': {'lon': -108.1833, 'lat': 25.3667}, ...",2022-07-31 03:13:42,Angostura,MX,25.3667,-108.1833,82.00,72.0,10.0,3.76
464,"{'coord': {'lon': 118.7833, 'lat': 42.1167}, '...",2022-07-31 03:13:54,Mujiayingzi,CN,42.1167,118.7833,83.73,50.0,15.0,6.82
466,"{'coord': {'lon': 17.1369, 'lat': 42.9622}, 'w...",2022-07-31 03:13:58,Korčula,HR,42.9622,17.1369,76.62,68.0,0.0,7.02
470,"{'coord': {'lon': 10.85, 'lat': 30.3333}, 'wea...",2022-07-31 03:14:06,Nālūt,LY,30.3333,10.8500,78.46,28.0,0.0,10.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = best_cities_df.head(10).copy()

hotel_df["HOTEL_PLACEID"] = "NULL"
hotel_df["HOTEL_NAME"] = "NULL"
hotel_df["HOTEL_CITY"] = "NULL"
hotel_df["HOTEL_COUNTRY"] = "NULL"

hotel_df

,JSON,DATE,CITY,COUNTRY,LAT,LNG,MAX_TEMP,HUMIDITY,CLOUDINESS,WIND_SPEED,HOTEL_PLACEID,HOTEL_NAME,HOTEL_CITY,HOTEL_COUNTRY
10,"{'coord': {'lon': -114.0611, 'lat': 27.9769}, ...",2022-07-31 02:58:37,Guerrero Negro,MX,27.9769,-114.0611,70.70,77.0,5.0,13.85,NULL,NULL,NULL,NULL
35,"{'coord': {'lon': -134.9692, 'lat': -23.1203},...",2022-07-31 02:59:30,Rikitea,PF,-23.1203,-134.9692,72.93,75.0,9.0,15.93,NULL,NULL,NULL,NULL
37,"{'coord': {'lon': -139.0333, 'lat': -9.8}, 'we...",2022-07-31 02:59:34,Atuona,PF,-9.8000,-139.0333,76.68,78.0,14.0,14.43,NULL,NULL,NULL,NULL
41,"{'coord': {'lon': -159.775, 'lat': -21.2078}, ...",2022-07-31 02:59:41,Avarua,CK,-21.2078,-159.7750,78.85,69.0,40.0,20.71,NULL,NULL,NULL,NULL
48,"{'coord': {'lon': -17.1, 'lat': 32.6667}, 'wea...",2022-07-31 02:59:55,Ponta do Sol,PT,32.6667,-17.1000,69.94,79.0,14.0,6.98,NULL,NULL,NULL,NULL
71,"{'coord': {'lon': -158.1547, 'lat': 21.3906}, ...",2022-07-31 03:00:40,Nānākuli,US,21.3906,-158.1547,83.34,67.0,1.0,14.65,NULL,NULL,NULL,NULL
75,"{'coord': {'lon': -149.8667, 'lat': -17.5667},...",2022-07-31 03:00:47,Haapiti,PF,-17.5667,-149.8667,79.72,66.0,14.0,22.12,NULL,NULL,NULL,NULL
88,"{'coord': {'lon': 18.8162, 'lat': -5.041}, 'we...",2022-07-31 03:01:12,Kikwit,CD,-5.0410,18.8162,74.08,40.0,32.0,0.65,NULL,NULL,NULL,NULL
105,"{'coord': {'lon': -89.0928, 'lat': 30.3674}, '...",2022-07-31 02:57:51,Gulfport,US,30.3674,-89.0928,73.74,71.0,1.0,8.32,NULL,NULL,NULL,NULL
121,"{'coord': {'lon': 93.0344, 'lat': 58.0728}, 'w...",2022-07-31 03:02:19,Strelka,RU,58.0728,93.0344,74.44,71.0,35.0,7.09,NULL,NULL,NULL,NULL


In [12]:
# params dictionary to update each iteration

params = {
    "location": "0,0",
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports

for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    
    lat = row["LAT"]
    lng = row["LNG"]

    # change location each iteration while leaving original params in place
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_data = requests.get(base_url, params=params)

    # convert to json
    hotel_json = hotel_data.json()
    # pprint(hotel_json)

#     # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.

    try:
        hotel_df.loc[index, "HOTEL_PLACEID"] = hotel_json["results"][0]["place_id"]
#         hotel_df.loc[index, "HOTEL_CITY"] = hotel_json["results"][0]["plus_code"]["compound_code"].split(",")[0].split(" ", 1)[1]
#         hotel_df.loc[index, "HOTEL_COUNTRY"] = hotel_json["results"][0]["plus_code"]["compound_code"].split(",")[1]
#         # print(f'HOTEL TO ADD: {hotel_json["results"][0]["name"]}')
        
    except (KeyError, IndexError):
        
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.


In [13]:
cleanhotel_df = hotel_df.loc[ (hotel_df["HOTEL_PLACEID"] != "NULL"), :]

cleanhotel_df

,JSON,DATE,CITY,COUNTRY,LAT,LNG,MAX_TEMP,HUMIDITY,CLOUDINESS,WIND_SPEED,HOTEL_PLACEID,HOTEL_NAME,HOTEL_CITY,HOTEL_COUNTRY
10,"{'coord': {'lon': -114.0611, 'lat': 27.9769}, ...",2022-07-31 02:58:37,Guerrero Negro,MX,27.9769,-114.0611,70.70,77.0,5.0,13.85,ChIJZ-lVpeL4M4ERoYGbJ6Zrglg,NULL,NULL,NULL
35,"{'coord': {'lon': -134.9692, 'lat': -23.1203},...",2022-07-31 02:59:30,Rikitea,PF,-23.1203,-134.9692,72.93,75.0,9.0,15.93,ChIJib4T7laIJ54RcJ1rUHQ9KXk,NULL,NULL,NULL
37,"{'coord': {'lon': -139.0333, 'lat': -9.8}, 'we...",2022-07-31 02:59:34,Atuona,PF,-9.8000,-139.0333,76.68,78.0,14.0,14.43,ChIJL4z7-PMhN3YRhdXuLcxVEH4,NULL,NULL,NULL
41,"{'coord': {'lon': -159.775, 'lat': -21.2078}, ...",2022-07-31 02:59:41,Avarua,CK,-21.2078,-159.7750,78.85,69.0,40.0,20.71,ChIJmVoxwbjRV3ERXkJfFo6QLjY,NULL,NULL,NULL
48,"{'coord': {'lon': -17.1, 'lat': 32.6667}, 'wea...",2022-07-31 02:59:55,Ponta do Sol,PT,32.6667,-17.1000,69.94,79.0,14.0,6.98,ChIJU0wF3-hZYAwRokB97zLZkqY,NULL,NULL,NULL
75,"{'coord': {'lon': -149.8667, 'lat': -17.5667},...",2022-07-31 03:00:47,Haapiti,PF,-17.5667,-149.8667,79.72,66.0,14.0,22.12,ChIJlQlAZRqImXYR6WtP_FwQONA,NULL,NULL,NULL
105,"{'coord': {'lon': -89.0928, 'lat': 30.3674}, '...",2022-07-31 02:57:51,Gulfport,US,30.3674,-89.0928,73.74,71.0,1.0,8.32,ChIJX363D8QXnIgRl4mdtez-kJM,NULL,NULL,NULL
121,"{'coord': {'lon': 93.0344, 'lat': 58.0728}, 'w...",2022-07-31 03:02:19,Strelka,RU,58.0728,93.0344,74.44,71.0,35.0,7.09,ChIJqUPbn68z0FwRYaXCDz1WF0M,NULL,NULL,NULL


In [14]:
# params dictionary to update each iteration

params = {
    "place_id": "",
    "key": g_key
}


for index, row in cleanhotel_df.iterrows():
    
    place_id = row["HOTEL_PLACEID"]

    # change location each iteration while leaving original params in place
    
    params["place_id"] = f"{place_id}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/details/json"

    # make request and print url
    address_data = requests.get(base_url, params=params)

    # convert to json
    address_json = address_data.json()
    # pprint(address_json)
    
    


    try:
        
#         hotel_city = address_json["result"]["formatted_address"]
#         hotel_country = address_json["result"]["formatted_address"]
#         hotel_name = address_json["result"]["formatted_address"]
        
        cleanhotel_df.loc[index, "HOTEL_CITY"] = address_json["result"]["formatted_address"]
#         hotel_df.loc[index, "HOTEL_CITY"] = hotel_json["results"][0]["plus_code"]["compound_code"].split(",")[0].split(" ", 1)[1]
#         hotel_df.loc[index, "HOTEL_COUNTRY"] = hotel_json["results"][0]["plus_code"]["compound_code"].split(",")[1]
#         # print(f'HOTEL TO ADD: {hotel_json["results"][0]["name"]}')
        
    except (KeyError, IndexError):
        
        print("Missing field/result... skipping.")

{'html_attributions': [],
 'result': {'address_components': [{'long_name': 'Paralelo 28',
                                    'short_name': 'Paralelo 28',
                                    'types': ['point_of_interest',
                                              'establishment']},
                                   {'long_name': 'Carretera Federal 1',
                                    'short_name': 'México 1',
                                    'types': ['route']},
                                   {'long_name': 'Guerrero Negro',
                                    'short_name': 'Guerrero Negro',
                                    'types': ['locality', 'political']},
                                   {'long_name': 'Baja California Sur',
                                    'short_name': 'B.C.S.',
                                    'types': ['administrative_area_level_1',
                                              'political']},
                                   {'long_na

{'html_attributions': [],
 'result': {'address_components': [{'long_name': 'French Polynesia',
                                    'short_name': 'PF',
                                    'types': ['country', 'political']},
                                   {'long_name': 'Gambier',
                                    'short_name': 'Gambier',
                                    'types': ['locality', 'political']},
                                   {'long_name': 'Îles Gambier',
                                    'short_name': 'Îles Gambier',
                                    'types': ['administrative_area_level_2',
                                              'political']},
                                   {'long_name': '98755',
                                    'short_name': '98755',
                                    'types': ['postal_code']}],
            'adr_address': 'BP 19 | Archipel des Gambier, Mangareva, <span '
                           'class="locality">Gambier</sp

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '5XWF+8MR',
                                    'short_name': '5XWF+8MR',
                                    'types': ['plus_code']},
                                   {'long_name': 'Atuona',
                                    'short_name': 'Atuona',
                                    'types': ['sublocality_level_1',
                                              'sublocality',
                                              'political']},
                                   {'long_name': 'Hiva Oa',
                                    'short_name': 'Hiva Oa',
                                    'types': ['locality', 'political']},
                                   {'long_name': 'Southern Group',
                                    'short_name': 'Southern Group',
                                    'types': ['administrative_area_level_2',
                                              'political']},
             

{'html_attributions': [],
 'result': {'address_components': [{'long_name': 'Ara Tapu',
                                    'short_name': 'Ara Tapu',
                                    'types': ['route']},
                                   {'long_name': 'Kaikaveka Tapere',
                                    'short_name': 'Kaikaveka Tapere',
                                    'types': ['sublocality_level_1',
                                              'sublocality',
                                              'political']},
                                   {'long_name': 'Avarua District',
                                    'short_name': 'Avarua District',
                                    'types': ['locality', 'political']},
                                   {'long_name': 'Cook Islands',
                                    'short_name': 'CK',
                                    'types': ['country', 'political']}],
            'adr_address': '<span class="street-address">Ara

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '10',
                                    'short_name': '10',
                                    'types': ['street_number']},
                                   {'long_name': 'VE3',
                                    'short_name': 'VE3',
                                    'types': ['route']},
                                   {'long_name': 'Ponta do Sol',
                                    'short_name': 'Ponta do Sol',
                                    'types': ['locality', 'political']},
                                   {'long_name': 'Madeira',
                                    'short_name': 'Madeira',
                                    'types': ['administrative_area_level_1',
                                              'political']},
                                   {'long_name': 'Portugal',
                                    'short_name': 'PT',
                                    'types': ['c

{'html_attributions': [],
 'result': {'address_components': [{'long_name': 'French Polynesia',
                                    'short_name': 'PF',
                                    'types': ['country', 'political']},
                                   {'long_name': 'Haapiti',
                                    'short_name': 'Haapiti',
                                    'types': ['sublocality_level_1',
                                              'sublocality',
                                              'political']},
                                   {'long_name': 'Moorea-Maiao',
                                    'short_name': 'Moorea-Maiao',
                                    'types': ['locality', 'political']},
                                   {'long_name': 'Îles du Vent',
                                    'short_name': 'Îles du Vent',
                                    'types': ['administrative_area_level_1',
                                              'politi

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '1600',
                                    'short_name': '1600',
                                    'types': ['street_number']},
                                   {'long_name': 'East Beach Boulevard',
                                    'short_name': 'E Beach Blvd',
                                    'types': ['route']},
                                   {'long_name': 'Gulfport',
                                    'short_name': 'Gulfport',
                                    'types': ['locality', 'political']},
                                   {'long_name': 'Harrison County',
                                    'short_name': 'Harrison County',
                                    'types': ['administrative_area_level_2',
                                              'political']},
                                   {'long_name': 'Mississippi',
                                    'short_name': 'MS',
       

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '12',
                                    'short_name': '12',
                                    'types': ['street_number']},
                                   {'long_name': 'Naberezhnaya Ulitsa',
                                    'short_name': 'Naberezhnaya Ulitsa',
                                    'types': ['route']},
                                   {'long_name': 'Strelka',
                                    'short_name': 'Strelka',
                                    'types': ['locality', 'political']},
                                   {'long_name': 'Yeniseysky District',
                                    'short_name': 'Yeniseysky District',
                                    'types': ['administrative_area_level_2',
                                              'political']},
                                   {'long_name': 'Krasnoyarskiy kray',
                                    'short_name'

In [15]:
cleanhotel_df

,JSON,DATE,CITY,COUNTRY,LAT,LNG,MAX_TEMP,HUMIDITY,CLOUDINESS,WIND_SPEED,HOTEL_PLACEID,HOTEL_NAME,HOTEL_CITY,HOTEL_COUNTRY
10,"{'coord': {'lon': -114.0611, 'lat': 27.9769}, ...",2022-07-31 02:58:37,Guerrero Negro,MX,27.9769,-114.0611,70.70,77.0,5.0,13.85,ChIJZ-lVpeL4M4ERoYGbJ6Zrglg,NULL,"Paralelo 28, México 1, 23940 Guerrero Negro, B...",NULL
35,"{'coord': {'lon': -134.9692, 'lat': -23.1203},...",2022-07-31 02:59:30,Rikitea,PF,-23.1203,-134.9692,72.93,75.0,9.0,15.93,ChIJib4T7laIJ54RcJ1rUHQ9KXk,NULL,"BP 19 | Archipel des Gambier, Mangareva, Gambi...",NULL
37,"{'coord': {'lon': -139.0333, 'lat': -9.8}, 'we...",2022-07-31 02:59:34,Atuona,PF,-9.8000,-139.0333,76.68,78.0,14.0,14.43,ChIJL4z7-PMhN3YRhdXuLcxVEH4,NULL,"5XWF+8MR, Hiva Oa, French Polynesia",NULL
41,"{'coord': {'lon': -159.775, 'lat': -21.2078}, ...",2022-07-31 02:59:41,Avarua,CK,-21.2078,-159.7750,78.85,69.0,40.0,20.71,ChIJmVoxwbjRV3ERXkJfFo6QLjY,NULL,"Ara Tapu, Avarua District, Cook Islands",NULL
48,"{'coord': {'lon': -17.1, 'lat': 32.6667}, 'wea...",2022-07-31 02:59:55,Ponta do Sol,PT,32.6667,-17.1000,69.94,79.0,14.0,6.98,ChIJU0wF3-hZYAwRokB97zLZkqY,NULL,"VE3 10, 9360 Ponta do Sol, Portugal",NULL
75,"{'coord': {'lon': -149.8667, 'lat': -17.5667},...",2022-07-31 03:00:47,Haapiti,PF,-17.5667,-149.8667,79.72,66.0,14.0,22.12,ChIJlQlAZRqImXYR6WtP_FwQONA,NULL,"PK 20 Vaianae Haapiti Moorea, Moorea-Maiao 987...",NULL
105,"{'coord': {'lon': -89.0928, 'lat': 30.3674}, '...",2022-07-31 02:57:51,Gulfport,US,30.3674,-89.0928,73.74,71.0,1.0,8.32,ChIJX363D8QXnIgRl4mdtez-kJM,NULL,"1600 E Beach Blvd, Gulfport, MS 39501, USA",NULL
121,"{'coord': {'lon': 93.0344, 'lat': 58.0728}, 'w...",2022-07-31 03:02:19,Strelka,RU,58.0728,93.0344,74.44,71.0,35.0,7.09,ChIJqUPbn68z0FwRYaXCDz1WF0M,NULL,"Naberezhnaya Ulitsa, 12, Strelka, Krasnoyarski...",NULL


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{HOTEL_NAME}</dd>
<dt>City</dt><dd>{HOTEL_CITY}</dd>
<dt>Country</dt><dd>{HOTEL_COUNTRY}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in df.iterrows()]
locations = df[["LAT", "LNG"]]

In [ ]:
# Add marker layer ontop of heat map

 # Assign the marker layer to a variable
    
markers = gmaps.marker_layer(locations=locations, display_info_box=True, info_box_content=hotel_info)

# Add the layer to the map

fig.add_layer(markers)
fig


# Display figure
